    Here I worked on the Bengali Language. Preproceesng work is done in separate notebook.

    Here I have worked with the whole Corpus, which is around 5 crore tokens.
    Here the task was to develop a language model using 3-grams.

###Importing Required Libraries

In [0]:
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
from nltk import bigrams, trigrams
import json
import time
from random import randint
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings("ignore")

### Mount Google Drive


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Reading the Bengali Corpus





In [0]:
## Reading the corpus
corpus=open("/content/drive/My Drive/Corpus.txt",encoding="utf-8").readlines()
data=corpus[0]

In [0]:
len(data)  ## length of the whole string

206406578

In [0]:
## creating a list with all the tokens which are in the sequence as in corpus
splitted_data=data.split() # Working with all the tokens

###Creating a placeholder for model

In [0]:
model = defaultdict(lambda: defaultdict(lambda: 0))

###Co-occurance frequency

In [0]:
## Stroing the frequency for a word to occur after given two words
for w1, w2, w3 in trigrams(splitted_data, pad_right=True, pad_left=True):
    model[(w1, w2)][w3] += 1

### Converting the frequencies into probabilities

In [0]:
for w1_w2 in model:
    ## Total count of words occuring after two given words 
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        ## Convert into probabilities
        model[w1_w2][w3] /= total_count


### Function to get the next most probable word given the previous two words




In [0]:
## We want to get the next most probable word given the previous two words
def next_max_probable_word(dic):
  value=[0]     ##probability value
  for i in dic.keys():
    if dic[i]>value[0]:
      next_word,value=[i],[dic[i]]    ## key and Probability value getting updated if a key with more probability is encountered 
    ## In case more than one word exists with maximum probability 
    elif dic[i]==value[0]: 
      value.append(dic[i])
      next_word.append(i)
    else:
      pass 
  ## next_word can be a list in case more than one word exists with maximum probability 
  ## Choose randomly among words with same maximum probability   
  if len(next_word)>1:
    r = randint(0, len(next_word)-1)   
    return next_word[r] 
  return next_word[0]

## Examples to check how the function works


In [0]:
print('["গ্রেপ্তার","করেন"]:',next_max_probable_word(model["গ্রেপ্তার","করেন"]))
print('["বাংলাদেশ", "জাতীয়তাবাদী"]:',next_max_probable_word(model["বাংলাদেশ", "জাতীয়তাবাদী"]))

["গ্রেপ্তার","করেন"]: এবং
["বাংলাদেশ", "জাতীয়তাবাদী"]: দলের


### Function for Generating Sentence





In [0]:
## The function takes a text and length(length of the sentence we want) and generates the sentence 

def generate_sentence(text,length):
  sentence=text+" "
  ## Given a text we take the last two words and based on that the next words are generated 
  word_1=sentence.split()[-2]
  word_2=sentence.split()[-1]
  while length>=0:
    try:
        ## Using the function to generate the next max probable word
        generated_word= next_max_probable_word(model[word_1, word_2])
        sentence += generated_word+ " "
        word_1 = word_2
        word_2 = generated_word
        length -= 1
    except:
        break
  return sentence      

###Some examples of generated setences

In [0]:
generate_sentence("বাঙালিদের উপর",25)    ## Generated sentence is not good

'বাঙালিদের উপর অত্যাচার শুরু করে দুইটি থাকে তাদের পয়েন্টে এবং পয়েন্টে দুইজন খেলোয়াড় তাদের গুটিসমূহ বিপরীত দিকে পয়েন্ট থেকে পর্যন্তকে বলা হয় মাঝপয়েন্ট অবস্থান্তর পয়েন্ট থেকে পর্যন্তকে বলা '

In [0]:
generate_sentence("মুক্ত নাগরিককে রাষ্ট্রীয় সিদ্ধান্ত",25)   ## Generated sentence is good

'মুক্ত নাগরিককে রাষ্ট্রীয় সিদ্ধান্ত নেয়ার ক্ষেত্রে এটিই মূল চেম্বার এখানে সরু হাঁটার রাস্তা সালের হিসাবে এটি ব্যবহার করা হয় তিনি একজন সাবাহ ছিলেন সাবাহ বিন জাবের সাবাহদের কূটনীতি প্রতিবেশি জাতিদের '

In [0]:
generate_sentence("জাতীয়তাবাদী দলের রাজনীতিবিদ",30)   ## Genrated sentence is moderately good

'জাতীয়তাবাদী দলের রাজনীতিবিদ এবং সাবেক সংসদ সদস্য নির্বাচিত হয়েছিলেন তিনি সালে বাংলাদেশ সরকার তাকে বীর প্রতীক খেতাব প্রদান করে জন্ম শিক্ষাজীবন কর্মজীবন রাজনৈতিক জীবন সালে তিনি তার প্রথম শ্রেণীর ক্রিকেটে অভিষেক ঘটে তার এরপর '

In [0]:
generate_sentence("উনবংশ শতাব্দীর",30)  ## Generated sentence is pretty good

'উনবংশ শতাব্দীর মধ্যভাগে তিব্বতের খাম্স অঞ্চলে অবস্থিত একটি প্রাচীন স্থাপনা এটি মূলত একটি ছোট শহর যেটি বাংলাদেশের টাঙ্গাইল জেলার অন্তর্গত একটি ইউনিয়ন আয়তন জনসংখ্যা অবস্থান সীমানা চট্টগ্রাম সিটি কর্পোরেশনের আওতাধীন একটি ইউনিয়ন আয়তন '

    Different types of examples are shown.The results are not satisfactory in some cases. Though in some cases the generated sentences are making sense and pretty good. But in some cases the results are awful.